In [1]:
from pathlib import Path

import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, Polygon
from valhalla import Actor, get_config


def get_isochrone(query: dict):
    result = {}
    isochrones = actor.isochrone(query)
    for contour_ix, isochrone in enumerate(isochrones["features"]):
        geom = isochrone["geometry"]["coordinates"]
        time = isochrone["properties"]["contour"]
        result[time] = Polygon(geom)
    return result


def get_walk_isochrones(location: dict[str, float], times: list[int] = [5, 10, 15, 20]):
    query = {
        "locations": [location],
        "costing": "pedestrian",
        "contours": [{"time": i} for i in times],
    }

    return get_isochrone(query)


def get_bike_isochrones(location: dict[str, float], times: list[int] = [10, 15, 30]):
    query = {
        "locations": [location],
        "costing": "bicycle",
        "contours": [{"time": i} for i in times],
    }

    return get_isochrone(query)


def get_auto_isochrones(location: dict[str, float], times: list[int] = [10, 15, 30]):
    query = {
        "locations": [location],
        "costing": "auto",
        "contours": [{"time": i} for i in times],
    }

    return get_isochrone(query)

In [2]:
# CITY = "budapest"
CITY = "madrid"

In [3]:
Path(f"../output/{CITY}").mkdir(parents=True, exist_ok=True)

In [4]:
config = get_config(
    tile_extract=f"../data/valhalla/{CITY}/valhalla_tiles.tar",
    verbose=True,
)

config["service_limits"]["isochrone"]["max_contours"] = 10
config["service_limits"]["isochrone"]["max_locations"] = 10_000
config["service_limits"]["isochrone"]["max_distance"] = 100_000

# instantiate Actor to load graph and call actions
actor = Actor(config)

2025/03/18 16:39:54.646871 [INFO] Tile extract successfully loaded with tile count: 37
2025/03/18 16:39:54.648050 [WARN] (stat): /data/valhalla/traffic.tar No such file or directory
2025/03/18 16:39:54.648065 [WARN] Traffic tile extract could not be loaded


In [5]:
stops = pd.read_csv(f"../data/stops/{CITY}/stops_with_centrality.csv", engine="pyarrow")
stops["geometry"] = stops.apply(lambda x: Point(x["stop_lon"], x["stop_lat"]), axis=1)
stops = gpd.GeoDataFrame(stops, geometry="geometry", crs=4326)
# stops.to_crs(CRS, inplace=True)
stops.head(3)

,Node,Eigenvector Centrality,Degree Centrality,Closeness Centrality,Betweenness Centrality,stop_id,clust,stop_lat,stop_lon,stop_name,geometry
0,3.0,0.000003,0.001158,0.061367,0.006125,5130,3,40.50730,-3.68606,Monasterio De Arlanza - Nuestra Señora De Valv...,POINT (-3.68606 40.5073)
1,3.0,0.000003,0.001158,0.061367,0.006125,5166,3,40.50767,-3.68597,Monasterio De Arlanza - Nuestra Señora De Valv...,POINT (-3.68597 40.50767)
2,690.0,0.000014,0.002703,0.065333,0.016070,3881,690,40.51320,-3.67918,Carretera De Alcobendas - Bercianos,POINT (-3.67918 40.5132)


In [6]:
locations = []
for row in stops.itertuples():
    locations.append({"lon": row.stop_lon, "lat": row.stop_lat})

In [7]:
isochones_wide = []
isochones = []
for row in stops.itertuples():
    w = get_walk_isochrones(
        {"lon": row.stop_lon, "lat": row.stop_lat}, times=[5, 10, 15]
    )
    b = get_bike_isochrones(
        {"lon": row.stop_lon, "lat": row.stop_lat}, times=[5, 10, 15]
    )
    isochones.append([row.stop_id, w[5], "walk", 5])
    isochones.append([row.stop_id, w[10], "walk", 10])
    isochones.append([row.stop_id, w[15], "walk", 15])
    isochones.append([row.stop_id, b[5], "bicycle", 5])
    isochones.append([row.stop_id, b[10], "bicycle", 10])
    isochones.append([row.stop_id, b[15], "bicycle", 15])
isochones_wide = pd.DataFrame.from_records(
    isochones_wide,
    columns=[
        "stop_id",
        "walk_5",
        "walk_10",
        "walk_15",
        "bike_5",
        "bike_10",
        "bike_15",
    ],
)
isochones = pd.DataFrame.from_records(
    isochones, columns=["stop_id", "geometry", "costing", "range"]
)

In [8]:
isochones.to_csv(f"../output/{CITY}/isochrones.csv", index=False)

isochrones_gdf = gpd.GeoDataFrame(isochones, geometry="geometry", crs=4326)
isochrones_gdf.to_file(f"../output/{CITY}/isochrones.geojson")

In [80]:
# isochones = pd.read_csv(f"../output/{CITY}/isochrones.csv")
# print(isochones.head(6).to_markdown(index=False))